In [3]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
import seaborn as sns
#from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Modelado y preprocesamiento
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,KFold

# Métricas
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    RocCurveDisplay,
    roc_auc_score
)



#from kmodes.kprototypes import KPrototypes

In [ ]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Adjust width for better display of many columns
pd.set_option('display.max_colwidth', None) # Display full content of cells

In [5]:
categorical_variables = ['phone_carrier','phone_disposable_validation','phone_ported_original_carrier',
                         'phone_ported_validation','email_deliverable_validation','email_disposable_validation',
                        'email_domain_free_provider_flag','email_is_breached_flag','website_exists']

In [6]:
model_data = pd.read_csv("data_clean.csv")

In [7]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9129 entries, 0 to 9128
Data columns (total 30 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   phone_carrier                                          9129 non-null   object 
 1   phone_disposable_validation                            9129 non-null   bool   
 2   phone_ported_original_carrier                          9129 non-null   object 
 3   phone_ported_validation                                9129 non-null   bool   
 4   phone_social_number_of_names_returned                  9129 non-null   int64  
 5   phone_social_number_of_photos_returned                 9129 non-null   int64  
 6   phone_social_registered_ecommerce_profiles             9129 non-null   int64  
 7   phone_social_registered_email_provider_profiles        9129 non-null   int64  
 8   phone_social_registered_messaging_profiles      

In [21]:
model_data.head()

,phone_carrier,phone_disposable_validation,phone_ported_original_carrier,phone_ported_validation,phone_social_number_of_names_returned,phone_social_number_of_photos_returned,phone_social_registered_ecommerce_profiles,phone_social_registered_email_provider_profiles,phone_social_registered_messaging_profiles,phone_social_registered_professional_profiles,phone_social_registered_profiles,phone_social_registered_social_media_profiles,email_deliverable_validation,email_disposable_validation,email_domain_free_provider_flag,email_is_breached_flag,email_number_of_breaches,email_social_number_of_names_returned,email_social_number_of_photos_returned,email_social_registered_consumer_electronics_profiles,email_social_registered_ecommerce_profiles,email_social_registered_email_provider_profiles,email_social_registered_entertainment_profiles,email_social_registered_messaging_profiles,email_social_registered_professional_profiles,email_social_registered_profiles,email_social_registered_social_media_profiles,email_social_registered_travel_profiles,website_exists,label
0,Comcel S.A. (Claro),False,Claro Comunicacion Celular S.A. (Comcel),True,0,0,1,0,1,1,5,2,True,False,True,True,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,6.0,1.0,0.0,True,0
1,Telefonica Colombia (Movistar),False,Movistar Colombia Telecomunicaciones S.A. ESP,False,0,1,0,0,1,0,1,0,True,False,True,True,5.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,8.0,4.0,0.0,True,0
2,Comcel S.A. (Claro),False,Claro Comunicacion Celular S.A. (Comcel),False,0,0,0,0,1,1,4,2,True,False,True,True,3.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,2.0,10.0,4.0,0.0,True,0
3,Comcel S.A. (Claro),False,Claro Comunicacion Celular S.A. (Comcel),False,0,1,0,0,1,0,4,3,True,False,True,True,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,6.0,2.0,0.0,True,0
4,Comcel S.A. (Claro),False,Claro Comunicacion Celular S.A. (Comcel),True,0,0,0,0,1,1,4,2,True,False,True,True,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,5.0,1.0,0.0,True,0


In [22]:
model_data['label'].value_counts()

label
0    8485
1     644
Name: count, dtype: int64

In [23]:
model_data['phone_carrier'].value_counts()

phone_carrier
Comcel S.A. (Claro)                                  5467
Colombia Movil (Tigo)                                1508
Telefonica Colombia (Movistar)                       1488
Partners Telecom (WOM)                                186
Virgin Mobile (Colombia)                              135
ALMACENES EXITO INVERSIONES S.A.S.                     99
EMPRESA DE TELECOMUNICACIONES DE BOGOTA                74
COMUNICACION CELULAR S.A.                              58
Telefnica Mviles Colombia (Movistar)                   22
Comcel                                                 18
COLOMBIA TELECOMUNICACIONES S.A. ESP                   15
Comunicacion Celular S.A.                              14
COLOMBIA MOVIL S.A. E.S.P.                             10
Colombia Mvil                                           7
Colombia Telecomunicaciones S.A. E.S.P.                 5
COMUNICACION CELULAR (Claro)                            4
Avantel S.A.                                            3


In [8]:
y = model_data['label']
model_data_train = model_data.drop('label',axis=1)

In [9]:
model_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9129 entries, 0 to 9128
Data columns (total 29 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   phone_carrier                                          9129 non-null   object 
 1   phone_disposable_validation                            9129 non-null   bool   
 2   phone_ported_original_carrier                          9129 non-null   object 
 3   phone_ported_validation                                9129 non-null   bool   
 4   phone_social_number_of_names_returned                  9129 non-null   int64  
 5   phone_social_number_of_photos_returned                 9129 non-null   int64  
 6   phone_social_registered_ecommerce_profiles             9129 non-null   int64  
 7   phone_social_registered_email_provider_profiles        9129 non-null   int64  
 8   phone_social_registered_messaging_profiles      

<h3> Catboost Classifier </h3>

In [10]:
X_train, X_test, y_train, y_test = train_test_split(model_data_train, y, test_size=0.2, random_state=42)

In [24]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7303, 29), (1826, 29), (7303,), (1826,))

In [25]:
sum(y_train)

518

In [26]:
sum(y_test)

126

In [11]:
pos_count = sum(y)

In [12]:
pos_count

644

In [13]:
neg_count = len(model_data_train) - pos_count

In [14]:
neg_count

8485

In [15]:
scale_pos_weight_value = neg_count / pos_count

print(f"Calculated scale_pos_weight: {scale_pos_weight_value}")

Calculated scale_pos_weight: 13.175465838509316


In [16]:
base_model = CatBoostClassifier(iterations=100,
                               depth=6,
                               learning_rate=0.1,
                               loss_function='Logloss',
                               cat_features=categorical_variables,
                               eval_metric='F1',
                               verbose=True,
                               scale_pos_weight=scale_pos_weight_value
                              )

In [17]:
base_model.fit(X_train, y_train)

0:	learn: 0.6570932	total: 70.9ms	remaining: 7.02s
1:	learn: 0.5943596	total: 94.1ms	remaining: 4.61s
2:	learn: 0.5923449	total: 113ms	remaining: 3.65s
3:	learn: 0.6136128	total: 137ms	remaining: 3.29s
4:	learn: 0.6197993	total: 143ms	remaining: 2.72s
5:	learn: 0.6144828	total: 162ms	remaining: 2.54s
6:	learn: 0.6222115	total: 173ms	remaining: 2.3s
7:	learn: 0.6289514	total: 189ms	remaining: 2.17s
8:	learn: 0.6230620	total: 200ms	remaining: 2.02s
9:	learn: 0.6143322	total: 211ms	remaining: 1.9s
10:	learn: 0.6189358	total: 220ms	remaining: 1.78s
11:	learn: 0.6221422	total: 237ms	remaining: 1.74s
12:	learn: 0.6280510	total: 251ms	remaining: 1.68s
13:	learn: 0.6238624	total: 272ms	remaining: 1.67s
14:	learn: 0.6275339	total: 281ms	remaining: 1.59s
15:	learn: 0.6266817	total: 297ms	remaining: 1.56s
16:	learn: 0.6346279	total: 312ms	remaining: 1.52s
17:	learn: 0.6364048	total: 320ms	remaining: 1.46s
18:	learn: 0.6523959	total: 333ms	remaining: 1.42s
19:	learn: 0.6610596	total: 347ms	remaini

In [18]:
# 4. Evaluate the model
y_pred = base_model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.61      0.74      1700
           1       0.08      0.44      0.13       126

    accuracy                           0.60      1826
   macro avg       0.51      0.53      0.44      1826
weighted avg       0.88      0.60      0.70      1826



In [19]:
feature_importances = base_model.get_feature_importance()

feature_importances_df = pd.DataFrame({'Feature': model_data_train.columns, 'Importance': feature_importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

print(feature_importances_df.head(20))

top_features = feature_importances_df.nlargest(10, 'Importance')['Feature']

                                                  Feature  Importance
2                           phone_ported_original_carrier   11.688295
10                       phone_social_registered_profiles   10.928113
25                       email_social_registered_profiles    9.649132
11          phone_social_registered_social_media_profiles    8.342622
16                               email_number_of_breaches    7.655931
21        email_social_registered_email_provider_profiles    6.791395
26          email_social_registered_social_media_profiles    6.585435
0                                           phone_carrier    6.344137
24          email_social_registered_professional_profiles    6.166218
20             email_social_registered_ecommerce_profiles    5.203487
15                                 email_is_breached_flag    4.597751
3                                 phone_ported_validation    3.509762
6              phone_social_registered_ecommerce_profiles    2.388198
5                  p

In [20]:
top_features

2                       phone_ported_original_carrier
10                   phone_social_registered_profiles
25                   email_social_registered_profiles
11      phone_social_registered_social_media_profiles
16                           email_number_of_breaches
21    email_social_registered_email_provider_profiles
26      email_social_registered_social_media_profiles
0                                       phone_carrier
24      email_social_registered_professional_profiles
20         email_social_registered_ecommerce_profiles
Name: Feature, dtype: object